# 1. Import Libraries

In [77]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, average_precision_score
import pickle
from sklearn.tree import DecisionTreeClassifier
from joblib import dump, load

# **Binary Classifiers: Problem 1**

# 2. Data Exploration

In [30]:
## read the data
df_data = pd.read_csv('/content/cicids_static_data.csv')

In [31]:
## check the 1st 5 rows
df_data.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,53,159,2,2,68,266,34,34,34.0,0.0,133,133,133.0,0.0,2.100629e+06,25157.232700,53.00000,90.066642,157,1,1,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0,0,0,40,40,12578.616350,12578.616350,34,133,73.6,54.224533,2940.3,0,0,0,0,0,0,0,0,1,92.00,34.0,133.0,40,0,0,0,0,0,0,2,68,2,266,-1,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,53,51673,2,2,90,166,45,45,45.0,0.0,83,83,83.0,0.0,4.954231e+03,77.409866,17224.33333,29827.358290,51666,3,4,4.0,0.0,4,4,3,3.0,0.0,3,3,0,0,0,0,64,64,38.704933,38.704933,45,83,60.2,20.813457,433.2,0,0,0,0,0,0,0,0,1,75.25,45.0,83.0,64,0,0,0,0,0,0,2,90,2,166,-1,-1,1,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,7496,43,1,1,0,6,0,0,0.0,0.0,6,6,6.0,0.0,1.395349e+05,46511.627910,43.00000,0.000000,43,43,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,40,20,23255.813950,23255.813950,0,6,2.0,3.464102,12.0,0,0,0,1,0,0,0,0,1,3.00,0.0,6.0,40,0,0,0,0,0,0,1,0,1,6,29200,0,0,40,0.0,0.0,0,0,0.0,0.0,0,0,ATTACK
3,46564,61,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,1.967213e+05,32786.885250,61.00000,0.000000,61,61,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,16393.442620,16393.442620,6,6,6.0,0.000000,0.0,0,0,0,0,1,1,0,0,1,9.00,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,440,307,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,80,69,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.000000e+00,28985.507250,69.00000,0.000000,69,69,69,69.0,0.0,69,69,0,0.0,0.0,0,0,0,0,0,0,40,0,28985.507250,0.000000,0,0,0.0,0.000000,0.0,0,0,0,0,1,0,0,0,0,0.00,0.0,0.0,40,0,0,0,0,0,0,2,0,0,0,251,-1,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [32]:
len(df_data)

25191

In [33]:
## some stats about the data
df_data.describe()
## From this we can drop the columns with only zeros: FWD

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,25191.000000,2.519100e+04,25191.000000,25191.00000,2.519100e+04,2.519100e+04,25191.000000,25191.000000,25191.000000,25191.000000,25191.00000,25191.000000,25191.000000,25191.000000,2.518500e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,25191.000000,25191.0,25191.000000,25191.0,2.519100e+04,2.519100e+04,2.519100e+04,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,2.519100e+04,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,2.519100e+04,25191.0,25191.0,25191.0,25191.0,25191.0,25191.0,25191.000000,2.519100e+04,25191.00000,2.519100e+04,25191.000000,25191.000000,25191.000000,25191.000000,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04
mean,8765.751379,1.302190e+07,14.372752,16.80628,5.837956e+02,2.992545e+04,219.933746,20.452860,62.091345,72.396546,624.78143,45.731333,231.531587,227.162493,inf,inf,1.139005e+06,2.295824e+06,6.924351e+06,1.629789e+05,1.269634e+07,2.327944e+06,2.246253e+06,6.762395e+06,1.163676e+06,1.002364e+07,1.843998e+06,1.244500e+06,4.274531e+06,1.092621e+06,0.048787,0.0,0.000079,0.0,3.314755e+02,3.722468e+02,5.995505e+04,6893.287051,18.223850,713.829423,139.739549,220.086914,2.957633e+05,0.026597,0.048787,0.000159,0.280378,0.302013,0.105831,0.000079,0.000159,0.687785,157.253539,62.091345,231.531587,3.314755e+02,0.0,0.0,0.0,0.0,0.0,0.0,14.372752,5.837956e+02,16.80628,2.992641e+04,7068.153348,2338.379024,10.750308,25.890358,8.175780e+04,4.553670e+04,1.631345e+05,5.662674e+04,6.095297e+06,3.628463e+05,6.366900e+06,5.795319e+06
std,19077.629319,3.191602e+07,1220.476270,1620.27646,7.757115e+03,3.632427e+06,739.251288,60.512982,191.647090,293.360718,1481.65387,70.516212,472.544033,624.163182,NaN,NaN,4.418710e+06,7.231620e+06,2.035185e+07,2.809065e+06,3.181629e+07,9.748498e+06,7.573190e+06,2.040783e+07,9.181693e+06,2.889146e+07,9.324723e+06,5.304017e+06,1.565741e+07,8.915008e+06,0.215427,0.0,0.008910,0.0,2.493288e+04,3.240846e+04,2.337848e+05,39174.193402,26.166304,1606.101337,252.870523,497.329648,1.240815e+06,0.160905,0.215427,0.012600,0.449193,0.459140,0.307628,0.008910,0.012600,0.666127,273.365361,191.647090,472.544033,2.493288e+04,0.0,0.0,0.0,0.0,0.0,0.0,1220.476270,7.757115e+03,1620.27646,3.632433e+06,14824.236344,9227.293545,1179.682409,6.436242,8.378623e+05,4.001228e+05,1.208254e+06,7.917691e+05,1.949681e+07,3.767660e+06,2.013473e+07,1.923531e+07
min,0.000000,-1.000000e+00,1.000000,0.00000,0.000000e+

In [34]:
## get names of columns of data
df_data.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

# 3. Data Cleaning (Remove NaNs and Infinities)

In [35]:
## get the columns with null values greater than zero
df_data.columns[df_data.isna().sum() >0]

Index(['Flow Bytes/s'], dtype='object')

In [36]:
## check if this column has really missing values
df_data['Flow Bytes/s'].isna().sum()

6

Here there was a problem in dropping the infinity values; even though I'd drop them they persist and keep being there: some reasons:
1.  There might be no nulls but 'np.inf' values
2.   Check from the describe if there are any inf values for different columns (as these won't appear in conditions of nulls)
3.   Drop by condition better than the index as this might be tricky; condition gets it at once with inplace = True
4.   I'm not sure if this affected but make a copy for the dataframe to drop on

In [37]:
## Drop nulls ==> this drops the nulls in flow/bytes
## Since the dataframe is of a quite large number we can easily drop the values ((we don't need to know the column or anything; the above trials was for us))
## But we need to remove it from the whole dataframe
df_data_copy = df_data.copy()
df_data_na = df_data_copy.dropna()

In [38]:
## length should be less by 6
len(df_data_na)

25185

In [39]:
# Here we drop the infinity values in Flow Bytes
df_data_na.drop(df_data_na[df_data_na['Flow Bytes/s'] == np.inf].index, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [40]:
## Recheck
idx = df_data_na.index[df_data_na['Flow Bytes/s'] == np.inf]
idx = list(idx)
print(idx, df_data_na['Flow Bytes/s'].isna().sum())

[] 0


In [41]:
## we can see the range of values (-1.2x10^-7 to 1.86*10^5) if we want to replace the NaN by mean value: the value was fixed
df_data_na['Flow Bytes/s'].describe()

count    2.517900e+04
mean     1.661845e+06
std      2.785234e+07
min     -1.200000e+07
25%      1.194084e+02
50%      4.713444e+03
75%      1.860465e+05
max      2.071000e+09
Name: Flow Bytes/s, dtype: float64

In [42]:
### Now we need to remove the infinity values in Flow Packets ==> we can see there are no inf values
idx2 = df_data_na.index[df_data_na[' Flow Packets/s'] == np.inf]
idx2

Int64Index([], dtype='int64')

In [43]:
df_data_na[' Flow Packets/s'].describe() ## And its describe is cleaned now

count    2.517900e+04
mean     6.680076e+04
std      2.420045e+05
min     -2.000000e+06
25%      6.514248e+00
50%      1.070754e+02
75%      2.209945e+04
max      3.000000e+06
Name:  Flow Packets/s, dtype: float64

In [44]:
## check the number of rows in dataframe
len(df_data_na)

25179

# 4. Split the data

In [45]:
## get the data
df_x = df_data_na.iloc[:, :-1]

In [46]:
## get the label of the data
df_y = df_data_na.iloc[:, -1]

In [47]:
X_train, X_test, y_train, y_test = train_test_split(
    df_x, df_y, test_size=0.3, random_state=42)

# 5. Classifier A (XGBoost)

In [48]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(X_train, y_train)

**In the next cells we calculate the metrics**

In [49]:
y_pred = clf.predict(X_test)

In [50]:
## Encoding to calculate Average precision Recall
y_pred_enc_f = []
y_test_enc_f = []
for i in range(len(y_pred)):
    if y_pred[i] == "BENIGN":
       y_pred_enc_f.append(0)
    else: 
       y_pred_enc_f.append(1)

    if y_test.iloc[i] == "BENIGN":
       y_test_enc_f.append(0)
    else: 
       y_test_enc_f.append(1)

In [51]:
average_precision = average_precision_score(y_test_enc_f, y_pred_enc_f)
print(average_precision)

0.9052908242099844


In [52]:
## Classification Report
c_report = classification_report(y_test, y_pred)
print(c_report)

              precision    recall  f1-score   support

      ATTACK       0.96      0.93      0.95       741
      BENIGN       0.99      1.00      0.99      6813

    accuracy                           0.99      7554
   macro avg       0.98      0.96      0.97      7554
weighted avg       0.99      0.99      0.99      7554



**Use this for f1 score of the class**

In [53]:
x = classification_report(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True )
f1_score_attack =  report['ATTACK']['f1-score'] 
f1_score_avg = report['macro avg']['f1-score'] 
## in general we should use the weighted avg for imbalanced, however since this considers that the positive class is the one with 
## minority instances, while the positive class here is considered as the 'Attack' class -maybe due to the alphabet- so this conflict makes us better choose the macro or the attack metrics

print(f1_score_avg)

0.9711883120011563


In [60]:
dump(clf, 'model_no_sc.bin', compress=True)

# 6. Classifier B (Decision Tree)

In [64]:
clf2 = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)

In [65]:
y_pred2 = clf2.predict(X_test)

In [66]:
c_report2 = classification_report(y_test, y_pred2)
print(c_report2)

              precision    recall  f1-score   support

      ATTACK       0.99      0.99      0.99       741
      BENIGN       1.00      1.00      1.00      6813

    accuracy                           1.00      7554
   macro avg       0.99      0.99      0.99      7554
weighted avg       1.00      1.00      1.00      7554



# **Multi-Classifiers: Problem 2**

# 2. Data Exploration

In [67]:
## read the data
df_data = pd.read_csv('iot_static_data.csv')

## check the 1st 5 rows
df_data.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,MI_dir_L0.1_mean,MI_dir_L0.1_variance,MI_dir_L0.01_weight,MI_dir_L0.01_mean,MI_dir_L0.01_variance,H_L5_weight,H_L5_mean,H_L5_variance,H_L3_weight,H_L3_mean,H_L3_variance,H_L1_weight,H_L1_mean,H_L1_variance,H_L0.1_weight,H_L0.1_mean,H_L0.1_variance,H_L0.01_weight,H_L0.01_mean,H_L0.01_variance,HH_L5_weight,HH_L5_mean,HH_L5_std,HH_L5_magnitude,HH_L5_radius,HH_L5_covariance,HH_L5_pcc,HH_L3_weight,HH_L3_mean,HH_L3_std,...,HH_jit_L0.01_weight,HH_jit_L0.01_mean,HH_jit_L0.01_variance,HpHp_L5_weight,HpHp_L5_mean,HpHp_L5_std,HpHp_L5_magnitude,HpHp_L5_radius,HpHp_L5_covariance,HpHp_L5_pcc,HpHp_L3_weight,HpHp_L3_mean,HpHp_L3_std,HpHp_L3_magnitude,HpHp_L3_radius,HpHp_L3_covariance,HpHp_L3_pcc,HpHp_L1_weight,HpHp_L1_mean,HpHp_L1_std,HpHp_L1_magnitude,HpHp_L1_radius,HpHp_L1_covariance,HpHp_L1_pcc,HpHp_L0.1_weight,HpHp_L0.1_mean,HpHp_L0.1_std,HpHp_L0.1_magnitude,HpHp_L0.1_radius,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc,Source,Class
0,19.570634,152.679121,39991.937640,20.937891,150.455024,38960.674060,23.168006,145.454142,36954.215040,33.115077,123.515434,27655.538890,62.619034,102.471926,15409.868150,19.570634,152.679121,39991.937640,20.937891,150.455024,38960.674060,23.168006,145.454142,36954.215040,33.115077,123.515434,27655.538890,62.619034,102.471926,15409.868150,18.885473,156.041505,202.780449,349.823034,1.602610e+05,-2.956340e+04,-0.370432,20.044037,154.488822,200.790831,...,30.034811,5.307252,1.298518e+02,4.880738,181.620841,227.548005,291.835506,59579.844560,17447.876,0.446623,4.927844,181.698241,227.581665,291.280119,59578.905890,17793.81865,0.455634,4.975746,181.767997,227.609424,290.719232,5.957503e+04,1.814181e+04,0.464725,4.997565,181.796886,227.619989,290.465036,5.957232e+04,1.829905e+04,0.468843,4.999756,181.799689,2.276210e+02,290.439555,5.957202e+04,1.831480e+04,4.692557e-01,Provision PT 838 Security Camera,BENIGN
1,49.893840,169.297843,54532.338720,62.957476,169.735104,54283.995040,82.340884,168.919514,53449.819730,98.214224,164.300338,51038.672810,124.460107,146.697496,42083.223280,49.893840,169.297843,54532.338720,62.957476,169.735104,54283.995040,82.340884,168.919514,53449.819730,98.214224,164.300338,51038.672810,124.460107,146.697496,42083.223280,49.886530,169.313860,233.534964,295.273638,9.040787e+04,-2.457803e+03,-0.039193,62.898830,169.837418,233.073744,...,97.549007,1.686998,6.319929e+01,1.980432,69.960478,3.999805,101.835497,15.998438,0.000,0.000000,1.988213,69.976286,3.999930,101.846358,15.999438,0.00000,0.000000,1.996055,69.992095,3.999992,101.857221,1.599994e+01,0.000000e+00,0.000000,1.999605,69.999210,4.000000,101.862109,1.600000e+01,0.000000e+00,0.000000,1.999960,69.999921,4.000000e+00,101.862598,1.600000e+01,0.000000e+00,0.000000e+00,Provision PT 737E Security Camera,BENIGN
2,1.996527,449.011775,409.365474,2.002395,448.141152,739.076602,2.166552,419.128740,11039.485060,4.621621,278.780846,34237.687910,29.401508,286.241737,32040.668620,1.996527,449.011775,409.365474,2.002395,448.141152,739.076602,2.166552,419.128740,11039.485060,4.621621,278.780846,34237.687910,29.401508,286.241737,32040.668620,1.996402,449.036043,19.999968,449.036043,3.999987e+02,0.000000e+00,0.000000,1.997840,449.021626,19.999988,...,14.978833,10.667405,2.061570e+02,1.996402,449.036043,19.999968,449.036043,399.998701,0.000,0.000000,1.997840,449.021626,19.999988,449.021626,399.999532,0.00000,0.000000,1.999279,449.007209,19.999999,449.007209,3.999999e+02,0.000000e+00,0.000000,2.428558,450.295009,20.081460,450.295009,4.032651e+02,0.000000e+00,0.000000,14.978833,455.354366,1.959295e+01,455.354366,3.838837e+02,0.000000e+00,0.000000e+00,Samsung SNH 1011 N Webcam,BENIGN
3,1.000000,60.000018,0.004849,1.000065,60.017569,4.743299,1.063813,76.195918,4110.590208,2.936

In [68]:
## get the columns with null values greater than zero
df_data.columns[df_data.isna().sum() >0]

### No missing: The data is clean but the data has categorical features

Index([], dtype='object')

# 3. Data Cleaning (from categorical to numeric)

In [69]:
######## Get dummies ##########

df_x = df_data.loc[: , df_data.columns != 'Class' ]
print(len(df_data.columns))

df_y = df_data.loc[: , df_data.columns == 'Class']

117


In [70]:
categ = df_x.dtypes == 'object'
idx = np.where(categ)[0]
col = df_x.iloc[:, idx]
col_names_categ = col.columns

In [71]:
categ_dummy = pd.get_dummies(col)
df_x.drop(columns = col_names_categ, inplace = True)
df_x = pd.concat([df_x, categ_dummy], axis = 1)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [72]:
len(df_x.columns)  ## 7 unique values in the source column

124

# 4. Split the Data

In [74]:
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    df_x, df_y, stratify = df_y, test_size=0.3, random_state=42)

# 5. Classifier A (XGBoost)

In [75]:
clf_m = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(X_train_m, y_train_m)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [78]:
y_pred_m = clf_m.predict(X_test_m)
c_report_m = classification_report(y_test_m, y_pred_m)
print(c_report_m)

                       precision    recall  f1-score   support

               BENIGN       1.00      1.00      1.00      6686
  gafgyt_combo_attack       0.00      0.00      0.00         6
   gafgyt_junk_attack       0.96      0.96      0.96       113
   gafgyt_scan_attack       0.97      0.93      0.95        60
    gafgyt_tcp_attack       1.00      0.97      0.98        64
    gafgyt_udp_attack       0.99      1.00      0.99       134
     mirai_ack_attack       0.88      1.00      0.94        29
    mirai_scan_attack       1.00      1.00      1.00        26
     mirai_syn_attack       1.00      0.97      0.98        30
     mirai_udp_attack       1.00      1.00      1.00       532
mirai_udpplain_attack       0.60      1.00      0.75         3

             accuracy                           1.00      7683
            macro avg       0.85      0.89      0.87      7683
         weighted avg       1.00      1.00      1.00      7683



In [79]:
dump(clf_m, 'model_multi.bin', compress=True)

['model_multi.bin']

# 6. Classifier B (Decision Tree)

In [80]:
clf2_m = DecisionTreeClassifier(random_state=0).fit(X_train_m, y_train_m)

In [81]:
y_pred2_m = clf2_m.predict(X_test_m)

In [ ]:
c_report2_m = classification_report(y_test_m, y_pred2_m)
print(c_report2_m)

# 7. Auxilary